In [ ]:
!pip install ultralytics


In [ ]:
!pip install roboflow

In [ ]:
!pip uninstall pytube

In [1]:
!pip install pytube

  Using cached pytube-15.0.0-py3-none-any.whl.metadata (5.0 kB)
Using cached pytube-15.0.0-py3-none-any.whl (57 kB)


In [2]:
!pip install --upgrade pytube

In [3]:
import cv2
from ultralytics import YOLO
from pytube import YouTube

# Download YouTube video
youtube_url = "https://www.youtube.com/watch?v=EFs9iXd33e4&t=3s"  # Replace with your YouTube video URL
video_path = "downloaded_video.mp4"

classes = ['swimmers', 'NonViolence', 'Violence', 'guns', 'knife', 'normal', 'violence']

yt = YouTube(youtube_url)
stream = yt.streams.filter(file_extension='mp4').first()
stream.download(filename=video_path)

# Load the model
model = YOLO("newbestv3.pt")
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"

# Get original video properties
original_w, original_h, original_fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Set desired FPS (increase if needed)
desired_fps = original_fps * 1  # Example: increasing the original FPS by a factor of 10

# Blur ratio (increase to make blurring effect stronger)
blur_kernel_size = 1703  # Example kernel size, adjust as needed

# Expansion margin for bounding box (e.g., 20 pixels)
expansion_margin = 70

# Video writer
video_writer = cv2.VideoWriter("object_blurring_output.avi", cv2.VideoWriter_fourcc(*"mp4v"), desired_fps, (original_w, original_h))

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break

    results = model.predict(im0, show=False)
    boxes = results[0].boxes.xyxy.cpu().tolist()
    clss = results[0].boxes.cls.cpu().tolist()

    if boxes is not None:
        for box, cls in zip(boxes, clss):
            # Expand the bounding box
            x1 = max(0, int(box[0]) - expansion_margin)
            y1 = max(0, int(box[1]) - expansion_margin)
            x2 = min(original_w, int(box[2]) + expansion_margin)
            y2 = min(original_h, int(box[3]) + expansion_margin)

            # Extract the object using the expanded bounding box coordinates
            obj = im0[y1:y2, x1:x2]
            # Apply blur to the object
            blur_obj = cv2.blur(obj, (blur_kernel_size, blur_kernel_size))
            # Replace the object in the original frame with the blurred version
            im0[y1:y2, x1:x2] = blur_obj

    cv2.imshow("ultralytics", im0)
    video_writer.write(im0)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
video_writer.release()
cv2.destroyAllWindows()



0: 320x640 1 NonViolence, 63.8ms
Speed: 0.0ms preprocess, 63.8ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 NonViolence, 80.3ms
Speed: 3.0ms preprocess, 80.3ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 NonViolence, 66.1ms
Speed: 0.0ms preprocess, 66.1ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 NonViolence, 83.1ms
Speed: 0.0ms preprocess, 83.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 NonViolence, 60.1ms
Speed: 3.4ms preprocess, 60.1ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 NonViolence, 74.2ms
Speed: 0.0ms preprocess, 74.2ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 NonViolence, 73.4ms
Speed: 0.0ms preprocess, 73.4ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 NonViolence, 63.7ms
Speed: 0.0ms preprocess, 63.7ms inference, 0.0ms 